# Simulação para a apresentação

In [1]:
import pickle
import numpy as np
import pandas as pd

In [2]:
def read_model(modelName):
    with open('modelos/' + modelName, 'rb') as pickle_file:
        content = pickle.load(pickle_file)
        return content

In [3]:
# Importando modelos
svm_small_gp = read_model('model_small_gp.pkl')
svm_md_gp = read_model('model_medium_gp.pkl')
svm_lg_gp = read_model('model_large_gp.pkl')
cluster_model = read_model('cluster_model.pkl')
se = read_model('standardScaler.pkl')

In [4]:
#### Variáveis Utilizadas em cada modelo
gp_medium_features = [
    'limiteEmpresaAnaliseCredito',
    'titulosEmAberto',
    'maiorAtraso',
    'percentualProtestos',
    'valorSolicitado',
    'valorAprovado',
]

gp_large_features = [
    'limiteEmpresaAnaliseCredito',
    'valorAprovado',
    'duplicatasAReceber',
    'titulosEmAberto',
    'valorSolicitado'
]

gp_small_features = [
    'faturamentoBruto',
    'periodoDemonstrativoEmMeses',
    'limiteEmpresaAnaliseCredito',
    'titulosEmAberto',
    'valorSolicitado',
    'valorAprovado',
    'capitalSocial'
]

k_means_features = [
 'margemBrutaAcumulada',
 'faturamentoBruto',
 'margemBruta',
 'percentualRisco',
 'percentualRisco',
 'intervaloFundacao'
]

In [5]:
def drop_columns_with_many_nulls(df):
    '''
    Colunas com mais de 40% de dados faltantes = drop
    df -> dataframe para ser ajustado
    '''
    percentage_of_nulls = df.isnull().sum() / df.shape[0]
    columns_to_drop = np.array(percentage_of_nulls[percentage_of_nulls > .4].index)
    adjusted_dataframe = df.drop(columns_to_drop, axis=1)
    return adjusted_dataframe

def initial_column_selection(df):
    '''
    ``Helper function``
    Dropa colunas que não serão utilizadas
    '''
    adjusted_df = drop_columns_with_many_nulls(df)
    
    ## Também não quero utilizar datas na minha regressão
    unused_columns = ['primeiraCompra']
    
    # Também não quero colunas que identifiquem o cliente, quero trata-los como um ponto único
    unused_columns = np.concatenate(
        (
            unused_columns, 
            np.array(['numero_solicitacao', 'cnpjSemTraco', 'nomeFantasia', 'razaoSocial', 'status'])
        )
    )
    
    return adjusted_df.drop(unused_columns, axis=1)

In [6]:
from sklearn.preprocessing import StandardScaler

def predict_group(model, df, normalizer, features_used=k_means_features):
    mapped_group_names = {0: 'medio', 1: 'pequeno', 2: 'grande'}
    initial_sample = df.sample(1).copy()
    test_sample = initial_sample[features_used].copy()
    
    total_nulls = test_sample.isnull().sum(axis=1).values[0]
    
    while total_nulls != 0:
        test_sample = test_sample.sample(1).copy()
        total_nulls = test_sample.isnull().sum(axis=1).values[0]
    
    sample_normalized = normalizer.transform(test_sample)
    y_pred = model.predict(sample_normalized)
    
    return (mapped_group_names[y_pred[0]], initial_sample)

In [7]:
def predict_value(model, sample, features_used):
    try:
        test_sample = sample[1][features_used].copy()
        features = test_sample.drop('valorAprovado', axis=1)
        y_true = test_sample['valorAprovado']
    
        log_test_sample = np.log(features + 1)
        y_pred = model.predict(log_test_sample)

        print(f'''
            Valor Real: {y_true.values[0]},
            Valor Predito: {np.exp(y_pred[0])}
        ''')
    except:
        print('Contém nulos')
    

In [13]:
df = pd.read_csv('../data/bootcamp-solicitacoescredito.csv')
df['intervaloFundacao'] = df['intervaloFundacao'].map({'De 0 a 5 anos': 0, 'De 6 a 10 anos': 1, 'De 11 a 16 anos': 2, 'Acima de 17 anos': 3})

sample = predict_group(cluster_model, df, se)

if sample[0] == 'pequeno':
    print('grupo pequeno')
    predict_value(svm_small_gp, sample, gp_small_features)
elif sample[0] == 'medio':
    print('grupo médio')
    predict_value(svm_md_gp, sample, gp_medium_features)
else:
    print('grupo Grande')
    predict_value(svm_lg_gp, sample, gp_large_features)

grupo médio

            Valor Real: 2000000.0,
            Valor Predito: 1804546.360800371
        
